In [28]:
import pandas as pd
import re
from itertools import chain  

## SQL Read

In [29]:
# Leer el archivo .sql
with open('Query_polizas_gs.sql', 'r', encoding='utf-8') as file:
    sql_query = file.read()

## Summary Tranformation 

In [30]:
# Define una expresión regular para encontrar la descripcion general de la query.
patron = r'--Summary:(.*?)Sources:'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
summary_docs = '\n'.join(coincidencias).strip()
summary_docs = summary_docs.replace('-', ' ')

# Resaltar en negrita algunas palabras en summary_docs y agregar iconos al principio
summary_docs = summary_docs.replace('Created Date:', '<i class="fas fa-calendar-alt icono"></i> <b>Created Date:</b>')
summary_docs = summary_docs.replace('Description:', '<i class="fa-solid fa-pen-to-square icono"></i> <b>Description:</b>')
summary_docs = summary_docs.replace('References:', '<i class="fas fa-book icono"></i> <b>References:</b>')   


## Source Transformation

In [34]:
# Define una expresión regular para encontrar la descripcion general de la query.
patron = r'--Sources:(.*?)Products:'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Concatenar las coincidencias en una sola cadena de texto
source_docs = '\n'.join(coincidencias).strip()
source_docs  = source_docs .replace('-', ' ')
source_docs  = source_docs .replace(' ', '')


print(source_docs )

rod_bronze.engage.cust_gestiones.
prod_bronze.mktinfo.cotizacion_vt7.
prod_gold_summarized.migs_upgraded.certificados
prod_bronze.vt7.qv_vt7_estructuragestion.
prod_gold_summarized.migs_upgraded.emision
hive_metastore.default.hsbc_presupuestos




## Comments Transformation

In [31]:
# Define una expresión regular para encontrar todo los comentarios
patron = r'(Step|LC)(.*?)--'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron, sql_query, re.DOTALL)

# Se guardar los comentarios en un df y en una lista. 
list_documentation = []
df_documentation = pd.DataFrame ()
df_documentation_html = pd.DataFrame ()

for coincidencia in coincidencias:
    list_documentation.append(str(coincidencia))


# Eliminar caracteres especiales de cada elemento de la lista
list_documentation = [re.sub(r"\)$", "", comentario, count=1) for comentario in list_documentation]
list_documentation = [re.sub(r"^\(", "", comentario, count=1) for comentario in list_documentation]
list_documentation = [re.sub(r"'", '', comentario) for comentario in list_documentation]

# Se guardan los comentarios en un DataFrame
df_documentation['comentarios'] = list_documentation
df_documentation_html['Comment'] = df_documentation['comentarios'].str.split(':').str[1]

# Se crea la campo Order que contiene el orden secuencial de los comentarios
df_documentation_html['Order'] = range(1, len(df_documentation) + 1)

# Se crea el campo Clase que identifica la clase a la que pertenece cada comentario
df_documentation_html['Clase'] = df_documentation['comentarios'].str.split(':').str[0]
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('_', '.')
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace('  ', ' ')
df_documentation_html['Clase'] = df_documentation_html['Clase'].str.replace(',', ' ')

# Funcion para identificar a que linea pertenece cada comentario
def line_count(cadena, sql):
    line_tmp = []
    line = [i + 1 for i, linea in enumerate(sql.splitlines()) if cadena in linea]
    line_tmp.append(line)
    return line_tmp

# Se crea el campo line_number ejecuntando la función line_count
line_number = []

for elemento in df_documentation_html['Comment']:
    line_number.append(line_count(elemento, sql_query)) 

line_number = list(chain(*line_number))

df_documentation_html['Line Number'] = line_number  

# Orden de las columnas del DataFrame
df_documentation_html = df_documentation_html[['Order', 'Clase', 'Comment', 'Line Number']]



## HTML Creation

In [32]:
import pandas as pd

# Convertir el DataFrame a HTML sin índice
table_html = df_documentation_html.to_html(index=False, escape=False)

# Agregar clases específicas a las columnas
table_html = table_html.replace("<th>Clase</th>", '<th class="Clase">Clase</th>')
table_html = table_html.replace("<th>Comment</th>", '<th class="Comment">Comment</th>')

# Agregar un título dentro de summary_docs
summary_docs = f"<h4 style='margin: 5px 0;'>Resumen de la consulta</h4>\n{summary_docs}"

# Agregar clases a las celdas de la columna "Comment"
for i in range(len(df_documentation_html['Comment'])):
    table_html = table_html.replace('<td>', '<td class="Comment">', 1)

# Definir el HTML completo con estilos CSS
html_string = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Process Tabla de Datos</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #2c2c2c;
            color: #d3d3d3;
            text-align: left;
            padding: 20px;
        }}
        h2 {{
            color: #ffffff;
        }}
        table {{
            margin: auto;
            border-collapse: collapse;
            background-color: #333;
            color: #d3d3d3;
            text-align: left;
            table-layout: auto;  /* Ajusta el ancho de las columnas al contenido */
        }}
        th, td {{
            border-left: none;
            border-right: none;
            border-top: 0.5px solid #555555;
            border-bottom: 0.5px solid #555555;
            padding: 10px;
            text-align: left;
            
        }}
        th {{
            background-color: #444;
        }}
        .Clase {{
            min-width: 100px; /* Ajusta el ancho de la columna Ciudad */
        }}
        .summary {{
            font-family: Arial, sans-serif;
            font-size: 17px; /* Aumenta el tamaño de la letra */
            background-color: #f9f9f9;
            color: #343434;
            text-align: left;
            margin: 10px 0 10px 0; /* Ajusta el margen superior e inferior */
            padding: 10px;
            border-radius: 10px; /* Redondea los bordes */
        }}
        .icono {{
            color: #32587e; /* Cambia el color del icono */
        }}
    </style>
</head>
<body>
    <h2>Summary</h2>
    <pre class="summary">{summary_docs}</pre>
    <h2>Process Comments</h2>
    {table_html}
</body>
</html>
"""

# Guardar en un archivo
with open("tabla.html", "w", encoding="utf-8") as f:
    f.write(html_string)

print("Archivo HTML generado con éxito.")


Archivo HTML generado con éxito.
